In [ ]:
!python -m pip install seaborn
!python -m pip install scikit-learn

In [ ]:
!python -m pip install "elasticsearch>=6.4.0,<7.0.0"

In [2]:
from elasticsearch import Elasticsearch
client = Elasticsearch("http://172.17.212.35:9200")
resp = client.search(index="retromotion-indexer_development_products",
                     body={"_source":["descriptions","descriptionsSource","nameSource","shortDescriptionSource","categoriesSource"],
                           "query": {"match_all": {}}})


In [3]:
print("Got %d Hits:" % resp['hits']['total'])

Got 22329 Hits:


In [5]:
import pandas as pd
import re
import numpy as np

import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
df_eng = pd.DataFrame(columns=['name','sdesc','category','catlevel0','catlevel1','catlevel2','catlevel3','catlevel4','catlevel5'])
df_de = pd.DataFrame(columns=['name','sdesc','desc','category','categorypath'])

In [7]:
def cleanStr(text):
   
    # remove numbers
    # no_number_string = re.sub(r'\d+','',lower_string)

    # remove all punctuation except words and space
    # text = re.sub(r'[^\w\s]','', lower_string)

    # clean = re.compile('<.*?>')

    # text = re.sub(clean,'',text)
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' .join(emoticons).replace('-', ''))

    return text


In [8]:
def readHtml(html):
    dftable = pd.read_html(html)
    return dftable

In [9]:
name_en=[]
name_de=[]

sdesc_en=[]
sdesc_de=[]

desc_en=[]
desc_de=[]

cat_en=[]
cat_de=[]

catpath_en=[]
catpath_de=[]

for hit in resp['hits']['hits']:
    list_row =[]
    for name in hit['_source']['nameSource']:
        if (name["language"]=="en"):
            list_row.append(cleanStr(name["value"]))
        else:
            name_de.append(name["value"])
    
    for sdesc in hit['_source']['shortDescriptionSource']:
        if (sdesc["language"]=="en"):
            list_row.append(cleanStr(sdesc["value"]))
        else:
            sdesc_de.append(sdesc["value"])

    # for desc in hit['_source']['descriptionsSource']:
    #     if (desc["language"]=="en"):
    #         desc_en.append(readHtml(desc["value"]))
    #     else:
    #         desc_de.append(desc["value"])
    
    for cats in hit['_source']['categoriesSource']:
        if (cats["language"]=="en"):
            list_row.append(cleanStr(cats["label"]))
        else:
            desc_de.append(cats["label"])

        if (cats["language"]=="en"):
            categories =cats["path"].split('/')
            i =0
            catls=[]
            for category in categories:

                list_row.append(cleanStr(category))
                i= i+1
            
        else:
            catpath_de.append(cats["path"])
    
    # new_row = pd.DataFrame({'name':_name, 'sdesc':_sdesc, 'category':_cat_en }, index=[0])

    # pd.concat([new_row,df_eng.loc[:]]).reset_index(drop=True)

    for i in range (0,9-len(list_row)):
        list_row.append(np.nan)

    df_eng.loc[len(df_eng)] = list_row



In [ ]:
# df_en = pd.DataFrame({'name':name_en,
#                        'sdesc':sdesc_en,
#                     #    'desc':desc_en,
#                        'category':cat_en,
#                        'categorypath':catpath_en})


# df_de = pd.DataFrame({'name':name_de,
#                        'sdesc':sdesc_de,
#                        'desc':desc_de,
#                        'category':cat_de,
#                        'categorypath':catpath_de})

In [10]:
df_eng.head()

,name,sdesc,category,catlevel0,catlevel1,catlevel2,catlevel3,catlevel4,catlevel5
0,denso dtm82363 thermostat coolant,thermostat coolant,thermostat,sparepart,cooling system,thermostat,NaN,NaN,NaN
1,zf 1043 010 297 automatic transmission,automatic transmission,automatic transmission,sparepart,transmission,automatic transmission,automatic transmission,NaN,NaN
2,zf 1060 040 003 automatic transmission,automatic transmission,automatic transmission,sparepart,transmission,automatic transmission,automatic transmission,NaN,NaN
3,vaico v10 0131 camshaft,camshaft,camshaft,sparepart,engine,engine timing,camshaft,NaN,NaN
4,vaico v26 0302 track control arm,track control arm,track control arm,sparepart,powertrain suspension,wheel suspension,track control arm,NaN,NaN


In [11]:
category = df_eng.groupby('category')

In [14]:
category.get_group('thermostat')

,name,sdesc,category,catlevel0,catlevel1,catlevel2,catlevel3,catlevel4,catlevel5
0,denso dtm82363 thermostat coolant,thermostat coolant,thermostat,sparepart,cooling system,thermostat,NaN,NaN,NaN


In [ ]:
df_eng.iloc[7]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
n_gram_vectorizer = CountVectorizer(ngram_range=(1, 1))
n_gram_vectorizer.fit(df_eng['name'] + df_eng['category'] )
transformed_vector_name = n_gram_vectorizer.transform(df_eng['name'])
transformed_vector_sdesc = n_gram_vectorizer.transform(df_eng['sdesc'])
transformed_vector_category = n_gram_vectorizer.transform(df_eng['category'])

# vocabulary


In [ ]:
n_gram_vectorizer.vocabulary_

In [ ]:
transformed_vector_name.shape

In [1]:
transformed_vector_name.toarray()

NameError: name 'transformed_vector_name' is not defined

In [ ]:

n_gram_vectorizer.vocabulary_.get('automatic')

In [ ]:
vocabulary.items()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, x_test, Y_train, y_test = train_test_split(transformed_vector_sdesc,
                                                    transformed_vector_category,
                                                    test_size=0.2,
                                                    random_state=0)

In [ ]:
X_train.shape, Y_train.shape

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [ ]:

Xtrain_ = torch.from_numpy(X_train).float()
Xtest_ = torch.from_numpy(x_test).float()
Xtrain_.shape
Ytrain_ = torch.from_numpy(Y_train).view(1,-1)[0]
Ytest_ = torch.from_numpy(y_test).view(1,-1)[0]
Ytrain_.shape

In [ ]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.fc2 = nn.Linear(hidden_size, hidden_size) 
        self.fc3 = nn.Linear(hidden_size, output_size) 
    
    def forward(self, x):
        x = F.sigmoid(self.fc1(x))
        x = F.sigmoid(self.fc2(x)) 
        x = self.fc3(x)
        
        return F.log_softmax(x, dim=-1)

In [ ]:
model = Net()

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

loss_fn = nn.NLLLoss()

In [ ]:
epoch_data = []
epochs = 1001

for epoch in range(1, epochs):

    optimizer.zero_grad()
    Ypred = model(Xtrain_)

    loss = loss_fn(Ypred , Ytrain_)
    loss.backward()

    optimizer.step()
        
    Ypred_test = model(Xtest_)
    loss_test = loss_fn(Ypred_test, Ytest_)
    
    _,pred = Ypred_test.data.max(1)
    
    accuracy = pred.eq(Ytest_.data).sum().item() / y_test.values.size
    epoch_data.append([epoch, loss.data.item(), loss_test.data.item(), accuracy])
    
    if epoch % 100 == 0:
        print ('epoch - %d (%d%%) train loss - %.2f test loss - %.2f accuracy - %.4f'\
               % (epoch, epoch/150 * 10 , loss.data.item(), loss_test.data.item(), accuracy))

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(token_pattern=u'(?ui)\b\w*[a-z]+\w*\b')
text = ["This is v000 Sparta!"]
tfidf_matrix =  tf.fit_transform(text)
feature_names = tf.get_feature_names_out() 
print(feature_names)


ValueError: empty vocabulary; perhaps the documents only contain stop words